In [7]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from extract import POLOCM2BASELINE
from utils import read_plan
from evaluator import ExecutabilityEvaluator

In [8]:
def test_exe(domain_name, trace, test_trace=None):
    debug = {
        # 'get_transition_sets': True,
        # 'extract_holes': True,
        # 'find_holes': True,
    }
    try:
        baseline = POLOCM2BASELINE(state_param=True, viz=False, debug=debug)
        model = baseline.extract_model([trace])
        
        actions = model.actions
        
        pddl_domain = model.to_pddl_domain("test_bw")
        evaluator = ExecutabilityEvaluator(pddl_domain, 'overall', debug=True)
        if test_trace is not None:
            action_trace = [step.action for step in test_trace]
            exe = evaluator.get_overall_executability('l', action_trace, set(), set())
        else:
            action_trace = [step.action for step in trace]
            exe = evaluator.get_overall_executability('l', action_trace, set(), set())
        if exe < 1:
            print(f"Warning! domain: {domain_name} - executability: {exe}")
        return exe
    except Exception as e:
        print(f"Error processing domain {domain_name}: {e}")
        return 0

In [9]:
def test_cross_exe(domain_name, po_trace, to_trace):
    debug = {}
    try:
        baseline = POLOCM2BASELINE(state_param=True, viz=False, debug=debug)
        model = baseline.extract_model([po_trace])
        learned_domain = model.to_pddl_domain(domain_name)
        gt_filename = os.path.join('../../data', 'goose-benchmarks', 'tasks', domain_name, 'domain.pddl')
        evaluator = ExecutabilityEvaluator(learned_domain, 'cross',gt_filename, debug=False)
        action_trace = [step.action for step in to_trace]
        exe = evaluator.get_cross_executabilities(action_trace)
        return exe
    except Exception as e:
        print(f"Error processing domain {domain_name}: {e}")
        return 0

In [10]:
test_plan = "(pickup b2?object),(stack b2?object b1?object),(pickup b3?object),(stack b3?object b2?object)"
test_trace = read_plan(test_plan)
po_trace = test_trace.to_partial_ordered_trace(0.3)

# test_exe("plan1", trace)
test_exe("plan2", po_trace)
# plan2 = "(pickup b2?object),(stack b2?object b1?object),(pickup b3?object),(stack b3?object b2?object)"
# test_exe("test_bw", read_plan(plan2))

1.0

In [11]:
test_plan = "(pickup b2?object),(stack b2?object b1?object),(pickup b3?object),(stack b3?object b2?object)"
test_trace = read_plan(test_plan)
po_trace = test_trace.to_partial_ordered_trace(0.3)


test_cross_exe("blocksworld", po_trace, test_trace)

(0.25, 0.7)

In [12]:
with open("../../data/plain_traces/plain_traces.txt", "r") as f:
    with open("res.csv", "w") as out:
        lines = f.readlines()
        for line in lines:
            try:
                details = line.split("&&")
                name = f"{details[0]}-{details[2]}-{details[3]}"
            
                plan = details[-1]
                trace = read_plan(plan)
                po_trace = trace.to_partial_ordered_trace(0.3)

                domain_name = details[0]
            
                exe_l, exe_gt = test_cross_exe(domain_name, po_trace, trace)
            except Exception as e:
                print(f"Error processing task {name}: {e}")
                exe_l, exe_gt = 0, 0
            print(f"Executability for {name}: {exe_l}, {exe_gt}")
            out.write(f"{name},{exe_l}, {exe_gt}\n")
            out.flush()


        

Executability for blocksworld-p01.pddl-easy: 0.2, 0.0
Executability for blocksworld-p02.pddl-easy: 0.05, 0.0
Executability for blocksworld-p03.pddl-easy: 0.35, 0.0
Executability for blocksworld-p04.pddl-easy: 0.15, 0.0
Executability for blocksworld-p05.pddl-easy: 0.225, 1.0
Executability for blocksworld-p06.pddl-easy: 0.125, 0.8
Executability for blocksworld-p07.pddl-easy: 0.1875, 0.295
Executability for blocksworld-p08.pddl-easy: 0.1875, 0.26666666666666666
Executability for blocksworld-p09.pddl-easy: 0.475, 0.0
Executability for blocksworld-p10.pddl-easy: 0.45, 0.0
Executability for blocksworld-p11.pddl-easy: 0.39, 0.35
Executability for blocksworld-p12.pddl-easy: 0.12999999999999998, 0.5
Executability for blocksworld-p13.pddl-easy: 0.24166666666666664, 0.4416666666666666
Executability for blocksworld-p14.pddl-easy: 0.2916666666666667, 0.39047619047619053
Executability for blocksworld-p15.pddl-easy: 0.1333333333333333, 0.33999999999999997
Executability for blocksworld-p16.pddl-easy: 

KeyboardInterrupt: 